# Web Scraping: Selenium

- Bizler internette bir web sitesine bağlanıp belirli alanlara tıklayarak geziniyor, bazı formlar dolduruyor veya butonlara tıklayarak bazı yeni ekranların karşımıza gelmesini bekliyoruz. İşte **Selenium**, bilgisayarımıza yükleyeceğiniz bir driver yardımıyla ekrana bir tarayıcı açarak, gerçek bir insan gibi istediğimiz işlemleri çalıştırmanızı sağlayan bir araçtır. 

## Dokümantasyon

- [Selenium](https://selenium-python.readthedocs.io/)

## Kütüphane Yükleme

```python
- !pip install selenium
```

## WebDriver Yükleme

Selenium kütüphanesini kullabilmek için bilgisayarlarımıza bir webdriver yüklememiz gerekiyor. Eğer imkanımız varsa **ChromeDriver** bizim için en iyi seçenek olacaktır ancak diğer tarayıcıların driverlarını da kullanabiliriz. Yükleme adımları için aşağıdaki yönergeyi takip edebilirsiniz:

1. Kullandığınız tarayıcınızın versiyonunu öğrenin. (Chrome > About Google Chrome)
2. [Linkteki](https://chromedriver.chromium.org/downloads) adrese giderek tarayıcınızın versiyonuyla uyumlu olan ChromeDriver'ı indirin.
3. İşletim sisteminizle uyumlu olan Driverı indirdikten sonra indirdiğiniz **chromedriver.exe** dosyasını Zip'ten çıkarıp uygun gördüğünüz bir dizine kopyalayın.

In [1]:
from bs4 import BeautifulSoup
import requests
import time, os

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import warnings
warnings.filterwarnings(action='ignore')

# Bilgisayarımızda chromedriver.exe dosyasını kaydettiğimiz dizin
chromedriver = "/Users/ataozarslan/Documents/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

## Örnek 1 - YouTube

### Dinamik Sayfalar

Bazı sayfalar içeriklerini dinamik olarak sunarlar, bu da tarayıcıya her yüklendiklerinde farklı görünebilecekleri anlamına gelir. Tarayıcılarımızdaki öğeleri inceleyerek gördüğünüz HTML, erişim anında oluşturulduğundan dolayı `Requests` ve `BeautifulSoup` kütüphaneleri istediğimiz sonucu veremeyebilir.

In [3]:
youtube_query = "https://www.youtube.com/results?search_query=how+to+become+a+data+scientist"

In [4]:
page = requests.get(youtube_query).text
soup = BeautifulSoup(page, "html.parser")

In [5]:
soup

<!DOCTYPE html>
<html darker-dark-theme="" darker-dark-theme-deprecate="" lang="tr-TR" style="font-size: 10px;font-family: Roboto, Arial, sans-serif;" system-icons="" typography="" typography-spacing=""><head><script data-id="_gd" nonce="P-DLOem0-yl-qwkKTygPOw">window.WIZ_global_data = {"MuJWjd":false,"nQyAE":{},"oxN3nb":{"1":false}};</script><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="ApvK67ociHgr2egd6c2ZjrfPuRs8BHcvSggogIOPQNH7GJ3cVlyJ1NOq/COCdj0+zxskqHt9HgLLETc8qqD+vwsAAABteyJvcmlnaW4iOiJodHRwczovL3lvdXR1YmUuY29tOjQ0MyIsImZlYXR1cmUiOiJQcml2YWN5U2FuZGJveEFkc0FQSXMiLCJleHBpcnkiOjE2OTUxNjc5OTksImlzU3ViZG9tYWluIjp0cnVlfQ==" http-equiv="origin-trial"/><script nonce="P-DLOem0-yl-qwkKTygPOw">var ytcfg={d:function(){return window.yt&&yt.config_||ytcfg.data_||(ytcfg.data_={})},get:function(k,o){return k in ytcfg.d()?ytcfg.d()[k]:o},set:function(){var a=arguments;if(a.length>1)ytcfg.d()[a[0]]=a[1];else{var k;for(k in a[0])ytcfg.d()[k]=a[0][k]}}};
window.ytcfg.set('EME

İlk olarak video başlıklarına ulaşmayı deneyelim.

In [6]:
soup.find("h3",{"class":"title-and-badge style-scope ytd-video-renderer"})

In [7]:
soup.find("div",{"id":"contents"})

In [8]:
soup.find("div",{"id":"container"})

<div id="container"><div id="related"><div class="autoplay skeleton-light-border-bottom"><div class="skeleton-bg-color" id="upnext"></div><div class="video-skeleton"><div class="video-details"><div class="thumbnail skeleton-bg-color"></div><div class="details flex-1"><div class="video-title text-shell skeleton-bg-color"></div><div class="video-meta text-shell skeleton-bg-color"></div></div></div></div></div><div class="video-skeleton"><div class="video-details"><div class="thumbnail skeleton-bg-color"></div><div class="details flex-1"><div class="video-title text-shell skeleton-bg-color"></div><div class="video-meta text-shell skeleton-bg-color"></div></div></div></div><div class="video-skeleton"><div class="video-details"><div class="thumbnail skeleton-bg-color"></div><div class="details flex-1"><div class="video-title text-shell skeleton-bg-color"></div><div class="video-meta text-shell skeleton-bg-color"></div></div></div></div><div class="video-skeleton"><div class="video-details">

Yaptığımız onca denemeye rağmen BeautifulSoup yardımıyla almak istediğimiz bilgilere bir türlü ulaşamadık.

> **SORU**: Bu neden olmuş olabilir?

Seçeneklerden biri; sayfayı önce Selenium ile yüklemek, sonrasında sayfanın HTML'ine BeautifulSoup ile erişmektir. Bunun için öncelikle ChromeDriver aracılığıyla YouTube arama sayfasını başlatıyoruz. 

> **NOT:** Bu işlemden sonra ekranınızda yeni bir tarayıcı açılmalıdır. Selenium'u kullanmaya devam edebilmemiz için bu pencereyi açık tutmamız gerekiyor!

In [ ]:
driver = webdriver.Chrome(chromedriver)
driver.get(youtube_query)

- Şimdi, kullandığımız WebDriver yardımıyla tekrardan kaynak kodumuza erişmeye çalışalım.

In [ ]:
driver.page_source

In [11]:
soup = BeautifulSoup(driver.page_source, "html.parser")

Ve tekrardan `BeautifulSoup` kullanarak videoların başlıklarına ulaşmaya çalışalım.

In [12]:
soup.find("h3",{"class":"title-and-badge style-scope ytd-video-renderer"})

<h3 class="title-and-badge style-scope ytd-video-renderer">
<ytd-badge-supported-renderer class="style-scope ytd-video-renderer" collection-truncate="" disable-upgrade="" hidden="">
</ytd-badge-supported-renderer>
<a aria-label="FASTEST Way to Learn Data Science and ACTUALLY Get a Job by Power Couple 9 months ago 9 minutes 67,250 views" class="yt-simple-endpoint style-scope ytd-video-renderer" href="/watch?v=AI1eKN1Eldg&amp;pp=ygUeaG93IHRvIGJlY29tZSBhIGRhdGEgc2NpZW50aXN0" id="video-title" title="FASTEST Way to Learn Data Science and ACTUALLY Get a Job">
<yt-icon class="style-scope ytd-video-renderer" hidden="" id="inline-title-icon"><!--css-build:shady--></yt-icon>
<yt-formatted-string aria-label="FASTEST Way to Learn Data Science and ACTUALLY Get a Job by Power Couple 9 months ago 9 minutes 67,250 views" class="style-scope ytd-video-renderer">FASTEST Way to Learn Data Science and ACTUALLY Get a Job</yt-formatted-string>
</a>
</h3>

In [13]:
soup.find("h3",{"class":"title-and-badge style-scope ytd-video-renderer"}).text.strip()

'FASTEST Way to Learn Data Science and ACTUALLY Get a Job'

In [14]:
soup.find("div", {"id":"contents"}).findAll("a",{"id":"video-title"})

[<a aria-label="FASTEST Way to Learn Data Science and ACTUALLY Get a Job by Power Couple 9 months ago 9 minutes 67,250 views" class="yt-simple-endpoint style-scope ytd-video-renderer" href="/watch?v=AI1eKN1Eldg&amp;pp=ygUeaG93IHRvIGJlY29tZSBhIGRhdGEgc2NpZW50aXN0" id="video-title" title="FASTEST Way to Learn Data Science and ACTUALLY Get a Job">
 <yt-icon class="style-scope ytd-video-renderer" hidden="" id="inline-title-icon"><!--css-build:shady--></yt-icon>
 <yt-formatted-string aria-label="FASTEST Way to Learn Data Science and ACTUALLY Get a Job by Power Couple 9 months ago 9 minutes 67,250 views" class="style-scope ytd-video-renderer">FASTEST Way to Learn Data Science and ACTUALLY Get a Job</yt-formatted-string>
 </a>,
 <a aria-label="Data Science Career: (Is Becoming A Data Scientist ACTUALLY Worth It?) by Shane Hummus 1 year ago 13 minutes, 1 second 133,327 views" class="yt-simple-endpoint style-scope ytd-video-renderer" href="/watch?v=O9nf1CqjGzI&amp;pp=ygUeaG93IHRvIGJlY29tZSBhIGR

In [15]:
for title in soup.find("div", {"id":"contents"}).findAll("a",{"id":"video-title"}):
    print(title.text.strip())

FASTEST Way to Learn Data Science and ACTUALLY Get a Job
Data Science Career: (Is Becoming A Data Scientist ACTUALLY Worth It?)
Data Science Roadmap 2023 | Learn Data Science Skills in 6 Months
How I'd become a Data Analyst | If I had to start all over again
How to Become a Data Scientist in the UK
FASTEST Way to Become a Data Analyst and ACTUALLY Get a Job
Day In The Life of a REMOTE DATA ANALYST
Skills Needed to Become a Data Scientist
मेरी कहानी सुनकर मत लेना Data Science 👆 | @SessionWithSumit | Josh Talks Hindi
How to tell if a career in Data Analytics is right for you...
Should you become a Data Analyst to become a Data Scientist?
The Harsh Reality of Being a Data Scientist
Data Scientist in Germany 🇩🇪 |
How I Would Learn Data Science (If I Had to Start Over)
How I Would Learn Data Science in 2023? (If I could start over)
FASTEST Way to Become a Data Scientist and Actually Get a Job
How I Would Learn Data Science in 2022
Why you should not be a data scientist
My *Realistic* Data S

> **SORU**: Ama elimizde sadece 20 video başlığımız var? Aslında Youtube'da veri bilimiyle bir sürü video var?

### Sayfalardaki Etkileşimler

Selenium kullanarak normalde bizim sadece dışarıdan müdahalede bulunarak gerçekleştirebildiğimiz etkileşimler de kurabiliriz. Örneğin,

- Belli linkleri tıklama
- Boş hücrelere inputlar girme
- Kaydırma
- Sürekle-Bırak

Eğer daha fazla veri bilimi video başlığı istiyorsak, daha fazla videonun sayfa tarafından yüklenebilmesi için ekranın altına doğru kaydırma yapmamız gerekir.

In [16]:
for i in range(5):

    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);") # Kaydırma İşlemi
    
    time.sleep(2) # Her kaydırma işleminden sonra 2 saniye bekle

In [17]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [18]:
for title in soup.find("div", {"id":"contents"}).findAll("a",{"id":"video-title"}):
    print(title.text.strip())

FASTEST Way to Learn Data Science and ACTUALLY Get a Job
Data Science Career: (Is Becoming A Data Scientist ACTUALLY Worth It?)
Data Science Roadmap 2023 | Learn Data Science Skills in 6 Months
How I'd become a Data Analyst | If I had to start all over again
How to Become a Data Scientist in the UK
FASTEST Way to Become a Data Analyst and ACTUALLY Get a Job
Day In The Life of a REMOTE DATA ANALYST
Skills Needed to Become a Data Scientist
मेरी कहानी सुनकर मत लेना Data Science 👆 | @SessionWithSumit | Josh Talks Hindi
How to tell if a career in Data Analytics is right for you...
Should you become a Data Analyst to become a Data Scientist?
The Harsh Reality of Being a Data Scientist
Data Scientist in Germany 🇩🇪 |
How I Would Learn Data Science (If I Had to Start Over)
How I Would Learn Data Science in 2023? (If I could start over)
FASTEST Way to Become a Data Scientist and Actually Get a Job
How I Would Learn Data Science in 2022
Why you should not be a data scientist
My *Realistic* Data S

- Süper! ChromeDriver bizim dışarıdan müdahale etmemize gerek kalmadan kaydırma işlemlerini kendi gerçekleştirdi ve bu sayede artık elimizdeki daha çok video başlığı var :)

Peki, en baştan tamamen WebDriver'ımızın kontrolünde olan bir arama yapmak istesek bunu nasıl yapacağız?

In [19]:
from selenium.webdriver.common.by import By

# Arama çubuğunun XPATH'i
search_box = driver.find_element(By.XPATH, '//input[@id="search"]')

# Önceki yaptığımız aramayı temizleme
search_box.clear()

# Yeni arama oluşturma
search_box.send_keys("machine learning")

# Aramayı gerçekleştirme
search_box.send_keys(Keys.RETURN)

Çok güzel `machine learning` aramamızı youtube üzerinden gerçekleştirdik. Peki karşıma gelen videolara ekstra bir filtre uygulamak istersem?

In [20]:
# Filtreleme butonunun XPATH'İ
filter_button = driver.find_element(By.XPATH, '//*[@id="container"]/ytd-toggle-button-renderer/yt-button-shape/button/yt-touch-feedback-shape/div')
filter_button.click()

In [21]:
# 4 dakikadan kısa olan videoları filtreleme
short_video = driver.find_element(By.XPATH, '//div[contains(@title, "Search for Under 4 minutes")]')
short_video.click()

Artık sayfa kaynağını daha önce olduğu gibi `BeautifulSoup` ile ayrıştırabilir veya doğrudan istediğimiz içerikleri alabiliriz. Örneğin, ilk kısa machine learning videosunun bazı özellikleri almaya çalışalım.

In [22]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [23]:
for title in soup.find("div", {"id":"contents"}).findAll("a",{"id":"video-title"}):
    print(title.text.strip())

What is Machine Learning?
Basit Şekilde Makine Öğrenmesi Nedir? | Machine Learning Dersleri
What is Machine Learning?
Machine Learning Explained in 100 Seconds
J. Maya  - Machine Learning (Official Lyric Video)
Machine Learning In 2 Minutes | What Is Machine Learning? | Machine Learning Basics | Simplilearn
Deep Learning Cars
The Truth About Machine Learning and AI Revealed
Flight Delay Prediction For Aviation Industry Using Machine Learning
Machine Learning Explained
Python for data science and machine learning
What Is Machine Learning and How Does It Work? | Machine Learning Explained
Machine Learning on Arduino with TensorFlow Lite Micro
Multi-Agent Hide and Seek
Machine Learning: Making Sense of a Messy World
Machine Learning Microbit Makecode DF Robot Maqueen Husky Lens 2 Bots #2
Supervised vs. Unsupervised Machine Learning: What's the Difference?
I can't STOP reading these Machine Learning Books!
TensorFlow in 100 Seconds


Şimdi de video başlıklarına erişmeye çalışalım.

In [24]:
soup.find("div", {"id":"contents"}).findAll("a",{"id":"video-title"})

[<a aria-label="What is Machine Learning? by GCFLearnFree 5 years ago 2 minutes, 3 seconds 58,929 views" class="yt-simple-endpoint style-scope ytd-video-renderer" href="/watch?v=QghjaS0WQQU&amp;pp=ygUQbWFjaGluZSBsZWFybmluZw%3D%3D" id="video-title" title="What is Machine Learning?">
 <yt-icon class="style-scope ytd-video-renderer" hidden="" id="inline-title-icon"><!--css-build:shady--></yt-icon>
 <yt-formatted-string aria-label="What is Machine Learning? by GCFLearnFree 5 years ago 2 minutes, 3 seconds 58,929 views" class="style-scope ytd-video-renderer">What is Machine Learning?</yt-formatted-string>
 </a>,
 <a aria-label="Basit Şekilde Makine Öğrenmesi Nedir? | Machine Learning Dersleri by Mert Cobanov 2 years ago 3 minutes, 43 seconds 17,934 views" class="yt-simple-endpoint style-scope ytd-video-renderer" href="/watch?v=mL7QuOOWzkg&amp;pp=ygUQbWFjaGluZSBsZWFybmluZw%3D%3D" id="video-title" title="Basit Şekilde Makine Öğrenmesi Nedir? | Machine Learning Dersleri">
 <yt-icon class="styl

In [25]:
titles = []
for title in soup.find("div", {"id":"contents"}).findAll("a",{"id":"video-title"}):
    titles.append(title.text.strip())
    
titles

['What is Machine Learning?',
 'Basit Şekilde Makine Öğrenmesi Nedir? | Machine Learning Dersleri',
 'What is Machine Learning?',
 'Machine Learning Explained in 100 Seconds',
 'J. Maya  - Machine Learning (Official Lyric Video)',
 'Machine Learning In 2 Minutes | What Is Machine Learning? | Machine Learning Basics | Simplilearn',
 'Deep Learning Cars',
 'The Truth About Machine Learning and AI Revealed',
 'Flight Delay Prediction For Aviation Industry Using Machine Learning',
 'Machine Learning Explained',
 'Python for data science and machine learning',
 'What Is Machine Learning and How Does It Work? | Machine Learning Explained',
 'Machine Learning on Arduino with TensorFlow Lite Micro',
 'Multi-Agent Hide and Seek',
 'Machine Learning: Making Sense of a Messy World',
 'Machine Learning Microbit Makecode DF Robot Maqueen Husky Lens 2 Bots #2',
 "Supervised vs. Unsupervised Machine Learning: What's the Difference?",
 "I can't STOP reading these Machine Learning Books!",
 'TensorFlow

Sıra youtuberlarda :)

In [26]:
soup.find("div", {"id":"contents"}).findAll("a",{"class":"yt-simple-endpoint style-scope yt-formatted-string"})

[<a class="yt-simple-endpoint style-scope yt-formatted-string" dir="auto" href="/channel/UCvOJU-ubyUqxGSDRN7xK4Ng" spellcheck="false">neptune_ai</a>,
 <a class="yt-simple-endpoint style-scope yt-formatted-string" dir="auto" href="/@GCFLearnFree" spellcheck="false">GCFLearnFree</a>,
 <a class="yt-simple-endpoint style-scope yt-formatted-string" dir="auto" href="/@GCFLearnFree" spellcheck="false">GCFLearnFree</a>,
 <a class="yt-simple-endpoint style-scope yt-formatted-string" dir="auto" href="/@mertcobanov" spellcheck="false">Mert Cobanov</a>,
 <a class="yt-simple-endpoint style-scope yt-formatted-string" dir="auto" href="/@mertcobanov" spellcheck="false">Mert Cobanov</a>,
 <a class="yt-simple-endpoint style-scope yt-formatted-string" dir="auto" href="/@OxfordSparks" spellcheck="false">OxfordSparks</a>,
 <a class="yt-simple-endpoint style-scope yt-formatted-string" dir="auto" href="/@OxfordSparks" spellcheck="false">OxfordSparks</a>,
 <a class="yt-simple-endpoint style-scope yt-formatted

In [27]:
creators = []
for creator in soup.find("div", {"id":"contents"}).findAll("a",{"class":"yt-simple-endpoint style-scope yt-formatted-string"}):
    creators.append(creator.text.strip())
    
creators

['neptune_ai',
 'GCFLearnFree',
 'GCFLearnFree',
 'Mert Cobanov',
 'Mert Cobanov',
 'OxfordSparks',
 'OxfordSparks',
 'Fireship',
 'Fireship',
 'J. Maya',
 'J. Maya',
 'Simplilearn',
 'Simplilearn',
 'Samuel Arzt',
 'Samuel Arzt',
 'Tech With Tim',
 'Tech With Tim',
 'Mathavan asuran Creation',
 'Mathavan asuran Creation',
 'Harvard University',
 'Harvard University',
 'MONIBE',
 'MONIBE',
 'MATLAB',
 'MATLAB',
 'Arduino',
 'Arduino',
 'OpenAI',
 'OpenAI',
 'Google',
 'Google',
 'MakeCode and Hardware',
 'MakeCode and Hardware',
 'Eye on Tech',
 'Eye on Tech',
 'Nicholas Renotte',
 'Nicholas Renotte',
 'Fireship',
 'Fireship']

- Burada bir sorun var video içeriklerini paylaşan hesaplar karşımıza 2 defa gelmiş. Hadi bunun nedenini araştıralım.

In [28]:
creators = []
for creator in soup.find("div", {"id":"contents"}).findAll("a",{"class":"yt-simple-endpoint style-scope yt-formatted-string"}):
    if len(creators) % 2 == 0:
        creators.append(creator.text.strip())
    else:
        creators.append("Empty")

creators_updated = [value for value in creators if value != "Empty"]
creators_updated

['neptune_ai',
 'GCFLearnFree',
 'Mert Cobanov',
 'OxfordSparks',
 'Fireship',
 'J. Maya',
 'Simplilearn',
 'Samuel Arzt',
 'Tech With Tim',
 'Mathavan asuran Creation',
 'Harvard University',
 'MONIBE',
 'MATLAB',
 'Arduino',
 'OpenAI',
 'Google',
 'MakeCode and Hardware',
 'Eye on Tech',
 'Nicholas Renotte',
 'Fireship']

Neyse ki biz Python biliyoruz :) Şimdi de görüntülenme sayılarını elde etmeye çalışalım.

In [29]:
soup.find("div", {"id":"contents"}).findAll("span",{"class":"inline-metadata-item style-scope ytd-video-meta-block"})

[<span class="inline-metadata-item style-scope ytd-video-meta-block">1.3K views</span>,
 <span class="inline-metadata-item style-scope ytd-video-meta-block">58K views</span>,
 <span class="inline-metadata-item style-scope ytd-video-meta-block">5 years ago</span>,
 <span class="inline-metadata-item style-scope ytd-video-meta-block">17K views</span>,
 <span class="inline-metadata-item style-scope ytd-video-meta-block">2 years ago</span>,
 <span class="inline-metadata-item style-scope ytd-video-meta-block">11K views</span>,
 <span class="inline-metadata-item style-scope ytd-video-meta-block">189K views</span>,
 <span class="inline-metadata-item style-scope ytd-video-meta-block">177K views</span>,
 <span class="inline-metadata-item style-scope ytd-video-meta-block">No views</span>,
 <span class="inline-metadata-item style-scope ytd-video-meta-block">60K views</span>,
 <span class="inline-metadata-item style-scope ytd-video-meta-block">74K views</span>,
 <span class="inline-metadata-item st

- Sanırım burada da benzer bir durum var. Hadi bunu da istediğimiz gibi düzenleyelim.

In [30]:
views = []
for view in soup.find("div", {"id":"contents"}).findAll("span",{"class":"inline-metadata-item style-scope ytd-video-meta-block"}):
    if len(views) % 2 == 0:
        views.append(view.text.strip())
    else:
        views.append("Empty")

views_updated = [value for value in views if value != "Empty"]
views_updated

['1.3K views',
 '5 years ago',
 '2 years ago',
 '189K views',
 'No views',
 '74K views',
 '6 years ago',
 '1 year ago',
 '5 months ago',
 '1 year ago',
 '6 years ago',
 '4 days ago',
 '3 hours ago',
 '2 years ago',
 '54 minutes ago',
 '4 years ago',
 '3 years ago',
 '3 years ago',
 '7 years ago',
 '15 hours ago',
 '2 years ago',
 '3 months ago',
 '8 months ago']

In [31]:
dates = []
for date in soup.find("div", {"id":"contents"}).findAll("span",{"class":"inline-metadata-item style-scope ytd-video-meta-block"}):
    if len(dates) % 2 == 1:
        dates.append(date.text.strip())
    else:
        dates.append("Empty")

dates_updated = [value for value in dates if value != "Empty"]
dates_updated

['58K views',
 '17K views',
 '11K views',
 '177K views',
 '60K views',
 '463K views',
 '281K views',
 '58K views',
 '68K views',
 '9.8M views',
 '8.6K views',
 'No views',
 '63K views',
 'No views',
 '865K views',
 '59K views',
 '8.9M views',
 '739K views',
 'No views',
 '45K views',
 '18K views',
 '548K views']

> **NOT:** Eğer birden fazla element seçimi yapmak istiyorsak, `driver.find_elements(...)` fonksiyonunu kullanabiliriz, bu bize tüm eşleşen elementlere ait çıktıları döndürecektir.

Eğer WebDriver kullanarak yapacağımız işlemleri tamamladıysak, çıkmadan önce driverımızı kapatmayı unutmuyoruz :)

In [32]:
driver.close()

Tüm tarayıcı oturumlarının kapatıldığından emin olmak için de çıkış yapabiliriz.

In [33]:
driver.quit()